In [80]:
def match_pattern(T: str, P: str) -> int:
    """returns the first index where match occurs"""
    n, m = len(T), len(P)
    for i in range(n - m):
        k = 0
        while k < m and T[i + k] == P[k]:
            k += 1
        if k == m:
            return i
    return -1

    

In [81]:
T = "elephant"
P = "phan"

match_pattern(T, P)

3

In [333]:
options = [0, 0, 0]
from __future__ import annotations
from dataclasses import dataclass
from typing import List
@dataclass
class Cell:
    cost: int = 0
    parent: int = 0

    def __repr__(self):
        return f"cost={self.cost},\
        action={self.parent})"


def edit_distance_recursive(S: str, T: str, m, n):
    MATCH = 0
    INSERT = 1
    DELETE = 2
    options = dict.fromkeys([MATCH, INSERT, DELETE], 0)
    if m == 0:
        return n
    if n == 0:
        return m
    options[MATCH] = edit_distance_recursive(S, T, m-1, n-1) + match(S[m-1], T[n-1])
    options[INSERT] = edit_distance_recursive(S, T, m, n-1) + 1
    options[DELETE] = edit_distance_recursive(S, T, m-1, n) + 1

    return min(options.values())

def match(c: str, d: str):
    """match two characters c and d"""
    return 0 if c == d else 1

def edit_distance_DP(S: str, T: str, m: int, n: int):
    """Implements the edit distance via dynamic programming"""
    MATCH = 0
    INSERT = 1
    DELETE = 2
    options = dict.fromkeys([MATCH, INSERT, DELETE], 0)
    # initialize table
    table = [[Cell() for _ in range(n + 1)] for _ in range(m + 1)]

    # initialize first column in every row
    for idx, row in enumerate(table):
        if idx > 0:
            row[0].parent = DELETE
            row[0].cost = idx
        else:
            row[0].parent = -1
    
    # initialize first row in every column
    for idx, cell in enumerate(table[0]):
        if idx > 0:
            cell.parent = INSERT
            cell.cost = idx
        else:
            cell.parent = -1
    
    for i in range(1, m + 1):
        for j in range(1, n + 1):
            options[MATCH] = table[i-1][j-1].cost + match(S[i-1], T[j-1])
            options[INSERT] = table[i][j-1].cost + 1
            options[DELETE] = table[i-1][j].cost + 1

            table[i][j].cost, table[i][j].parent = min(zip(options.values(),options.keys()))

    # pd.DataFrame(table, columns=["empty_string"] + list(T), index=['empty_string'] + list(S))
    return table

def reconsruct_path(table: List[List[Cell]], S: str, T: str, m: int, n: int, result: List[str]):
    MATCH = 0
    INSERT = 1
    DELETE = 2
    if table[m][n].parent == '-1':
        return
    elif table[m][n].parent == MATCH:
        reconsruct_path(table, S, T, m-1, n - 1, result)
        if S[m-1] == T[n-1]:
            result.append("M")
        else:
            result.append("S")
    elif table[m][n].parent == INSERT:
        reconsruct_path(table, S, T, m, n-1, result)
        result.append("I")
    elif table[m][n].parent == DELETE:
        reconsruct_path(table, S, T, m-1, n, result)
        result.append("D")

    


In [346]:
S = "thou-shalt-not"
T = 'you-should-not'
m = len(S)
n = len(T)
#edit_distance_recursive(S, T, m, n)

In [347]:
table = edit_distance_DP(S, T, m, n)

In [348]:
df = pd.DataFrame(table, columns=["empty_string"] + list(T), index=['empty_string'] + list(S))

In [350]:
",".join(result)

'D,S,M,M,M,M,M,I,S,M,S,M,M,M,M'

In [363]:
MATCH = 0
INSERT = 1
DELETE = 2
MAPPING = {MATCH: 'M', INSERT: 'I', DELETE: 'D'}

In [364]:
cost = df.applymap(lambda x: x.cost)
action = df.applymap(lambda x: MAPPING.get(x.parent))

In [365]:
action
print("cost")
cost

,empty_string,y,o,u,-,s,h,o,u,l,d,-,n,o,t
empty_string,None,I,I,I,I,I,I,I,I,I,I,I,I,I,I
t,D,M,M,M,M,M,M,M,M,M,M,M,M,M,M
h,D,M,M,M,M,M,M,I,I,I,I,I,I,I,I
o,D,M,M,M,M,M,M,M,I,I,I,I,I,M,I
u,D,M,D,M,I,I,I,I,M,I,I,I,I,I,I
-,D,M,D,D,M,I,I,I,I,M,M,M,I,I,I
s,D,M,D,D,D,M,I,I,I,I,M,M,M,M,M
h,D,M,D,D,D,D,M,I,I,I,I,I,I,M,M
a,D,M,D,D,D,D,D,M,M,M,M,M,M,M,M
l,D,M,D,D,D,D,D,M,M,M,I,I,I,I,I


cost


,empty_string,y,o,u,-,s,h,o,u,l,d,-,n,o,t
empty_string,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
t,1,1,2,3,4,5,6,7,8,9,10,11,12,13,13
h,2,2,2,3,4,5,5,6,7,8,9,10,11,12,13
o,3,3,2,3,4,5,6,5,6,7,8,9,10,11,12
u,4,4,3,2,3,4,5,6,5,6,7,8,9,10,11
-,5,5,4,3,2,3,4,5,6,6,7,7,8,9,10
s,6,6,5,4,3,2,3,4,5,6,7,8,8,9,10
h,7,7,6,5,4,3,2,3,4,5,6,7,8,9,10
a,8,8,7,6,5,4,3,3,4,5,6,7,8,9,10
l,9,9,8,7,6,5,4,4,4,4,5,6,7,8,9


In [366]:
result = []
reconsruct_path(table, S, T, m, n, result)

In [367]:
result

['D', 'S', 'M', 'M', 'M', 'M', 'M', 'I', 'S', 'M', 'S', 'M', 'M', 'M', 'M']

In [456]:
# longest nondecreasing subsequence
def longest_subsequence(arr, type='non-decreasing'):
    n = len(arr)
    maximum_nd_so_far = 1
    maximum_inc_so_far = 1
    for j in range(n):
        length_of_inc_subsequence = 1
        length_of_ndc_subsequence = 1
        i = 0
        curr = arr[i]
        while i < j:
            if arr[j] > curr:
                length_of_inc_subsequence += 1
            if arr[j] >= curr:
                length_of_ndc_subsequence += 1
            i += 1
            curr = arr[i]
        maximum_nd_so_far = max(maximum_nd_so_far, length_of_ndc_subsequence)
        maximum_inc_so_far = max(maximum_inc_so_far, length_of_inc_subsequence)
    if type == 'non-decreasing':
        return maximum_nd_so_far
    return maximum_inc_so_far
        



In [444]:
longest_subsequence([-1, 3, 4, 5, 2, 2,2,2, 2, 2])

7

In [457]:
longest_subsequence([-1, 3, 4, 5, 2, 2,2,2, 2, 2], 'increasing')

4

In [448]:
def lsi(arr, n):
    def recurse(arr, i):
        if i == 0: return 1
        else:
            j = 0
            ans = 1
            while j < i:
                ans = max(ans, 1 + lsi(arr, j))
                j += 1
            return ans
    if n == 0:
        return 1
    ans = 1
    for i in range(n):
        ans = max(ans, recurse(arr, i))
    return ans



In [468]:
A = [10, 2, 5, 3, 7, 101, 18]
longest_subsequence(A, 'increasing')

6

In [469]:
lsi(A, len(A))

4